In [20]:
import os
import numpy as np
from PIL import Image
import random
from collections import defaultdict
from styles import styles, styles_count

# Скачиваем данные

In [ ]:
import os
import requests
import zipfile
from io import BytesIO

repo_url = "https://github.com/Skyfallk/2025_deep_gen_models"
folder_path = "HW_1.Bias_gen_and_autoencoders/avatars"

zip_url = f"{repo_url}/archive/main.zip"

response = requests.get(zip_url)
if response.status_code != 200:
    raise Exception("Не удалось скачать репозиторий")

with zipfile.ZipFile(BytesIO(response.content)) as zip_file:
    for file in zip_file.namelist():
        if file.startswith(f"2025_deep_gen_models-main/{folder_path}/"):
            local_path = os.path.join(*file.split("/")[2:])
            
            if file.endswith('/'):
                os.makedirs(local_path, exist_ok=True)
            else:
                os.makedirs(os.path.dirname(local_path), exist_ok=True)
                with open(local_path, 'wb') as f:
                    f.write(zip_file.read(file))

print(f"Папка avatars успешно скачана!")

Папка imgs успешно скачана!


## Генератор на основе данных о популярных стилях
MLE:

$P(\text{стиль}_i) = \frac{\text{количество стиля}_i}{\text{общее количество наблюдений}}$

In [ ]:
class SimpleStyleGenerator:
    def __init__(self):
        self.styles = styles
        self.styles_count = styles_count
        self.total_counts = {category: sum(counts) for category, counts in styles_count.items()}
        
    def generate_style(self):
        avatar_style = {}
        log_probability = 0.0
        
        for category in self.styles:
            options = self.styles[category]
            counts = self.styles_count[category]
            probabilities = np.array(counts) / self.total_counts[category] # MLE
            
            chosen_idx = np.random.choice(len(options), p=probabilities)
            chosen_style = options[chosen_idx]
            prob = probabilities[chosen_idx]
            
            avatar_style[category] = chosen_style
            log_probability += np.log(prob)
        
        probability = np.exp(log_probability)
        return avatar_style, probability

In [9]:
generator = SimpleStyleGenerator()
for _ in range(5):
    style, prob = generator.generate_style()
    print("Сгенерированный стиль:")
    for k, v in style.items():
        print(f"  {k}: {v}")
    print(f"Вероятность генерации: {prob:.6f}\n")

Сгенерированный стиль:
  прическа: нет волос
  цвет волос: рыжий
  аксесуар: солнцезащитные очки
  одежда: футболка с круглым вырезом
  цвет одежды: розовый
Вероятность генерации: 0.000463

Сгенерированный стиль:
  прическа: короткая прямые
  цвет волос: рыжий
  аксесуар: нет очков
  одежда: худи
  цвет одежды: красный
Вероятность генерации: 0.000152

Сгенерированный стиль:
  прическа: короткая прямые
  цвет волос: черный
  аксесуар: круглые очки
  одежда: футболка с круглым вырезом
  цвет одежды: белый
Вероятность генерации: 0.000618

Сгенерированный стиль:
  прическа: короткая прямые
  цвет волос: блонд
  аксесуар: нет очков
  одежда: футболка с круглым вырезом
  цвет одежды: серый
Вероятность генерации: 0.000265

Сгенерированный стиль:
  прическа: длинные прямые
  цвет волос: черный
  аксесуар: круглые очки
  одежда: футболка с круглым вырезом
  цвет одежды: синий
Вероятность генерации: 0.001077



## Пиксельный генератор стилей

мне кажется тут все не так и я не успеваю до дедлайна

In [ ]:
import pprint
class PixelAvatarGenerator:
    def __init__(self, folder):
        self.folder = folder
        self.pixel_probs = self._compute_pixel_distributions()
        pprint.pprint(self.pixel_probs[10][10][0])


    def _compute_pixel_distributions(self):
        # Собираем все изображения
        image_arrays = []
        for fname in os.listdir(self.folder):
            if fname.lower().endswith(('.png', '.jpg', '.jpeg')):
                try:
                    img = Image.open(os.path.join(self.folder, fname)).convert('RGB')
                    image_arrays.append(np.array(img))
                except Exception as e:
                    print(f"Ошибка при загрузке {fname}: {e}")
        
        if not image_arrays:
            raise ValueError("Не найдено подходящих изображений в папке")
        
        images = np.stack(image_arrays)
        h, w, c = images.shape[1:]
        
        # Добавим сглаживание Лапласа для избежания нулевых вероятностей
        laplace_alpha = 1
        
        pixel_probs = []
        for y in range(h):
            row = []
            for x in range(w):
                channel_probs = []
                for ch in range(c):
                    # Получаем уникальные значения и их частоты
                    values, counts = np.unique(images[:, y, x, ch], return_counts=True)
                    
                    # Создаем полное распределение (0-255) со сглаживанием
                    full_probs = np.ones(256) * laplace_alpha
                    for v, cnt in zip(values, counts):
                        full_probs[v] += cnt
                    
                    # Нормализуем
                    full_probs = full_probs / full_probs.sum()
                    
                    # Сохраняем как словарь для экономии памяти (только ненулевые значения)
                    prob_dict = {i: float(full_probs[i]) for i in np.where(full_probs > laplace_alpha/256)[0]}
                    channel_probs.append(prob_dict)
                row.append(channel_probs)
            pixel_probs.append(row)
        
        return pixel_probs

    def generate(self):
        h = len(self.pixel_probs)
        w = len(self.pixel_probs[0])
        avatar = np.zeros((h, w, 3), dtype=np.uint8)
        total_log_prob = 0.0

        for y in range(h):
            for x in range(w):
                for ch in range(3):
                    probs = self.pixel_probs[y][x][ch]
                    
                    if not probs:
                        avatar[y, x, ch] = 0
                        continue
                    
                    # Нормализуем вероятности
                    values, weights = zip(*probs.items())
                    weights = np.array(weights, dtype=np.float64)
                    weights /= weights.sum()
                    
                    # Выбираем значение
                    choice = np.random.choice(values, p=weights)
                    avatar[y, x, ch] = choice
                    total_log_prob += np.log(weights[values.index(choice)])
        
        return Image.fromarray(avatar), np.exp(total_log_prob)


In [35]:
generator = PixelAvatarGenerator(folder='avatars')
for i in range(5):
    avatar, prob = generator.generate()
    avatar.save(f'generated_avatar_{i}.png')
    print(f"Вероятность генерации аватара {i}: {prob:.10f}")

{0: 0.0449438202247191}
Вероятность генерации аватара 0: 0.0000000000
Вероятность генерации аватара 1: 0.0000000000
Вероятность генерации аватара 2: 0.0000000000
Вероятность генерации аватара 3: 0.0000000000
Вероятность генерации аватара 4: 0.0000000000
